## Start

In [13]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Partner

### Links: 
___

["id",                                                                                           
 "issue_id", - это на будущее,  когда будет группировка жалоб по ишьб                                                                                    
 "src_type", какая сущность пожаловалсь (Citizen/Camera/Intercom)                                                                             
 "src_uuid", - uuid этой сущности                                                                                 
 "src_address_uuid", - адрес этой сущности                                                                      
 "src_address_type",     адрес этой сущности                                                                        
 "device_uuid", - uuid устройтсва на которое жалуется сущность                                                                              
 "device_type",                                                                
 "err_code", код ошибки intercom_offline / video / no_intercom                                                                           
 "err_src", откуда пришло MOB (мобилка) / WEB (веб интерфейсы для партнеров) / API (бэк )/ DEV (устройство)                                                                              
 "err_func",  sync (там еще будет)                                                                                   
 "aasm_state",  - статус жалобы pending / processed / failed                                                                             
 "process_error", - ошибка если статус failed                                                                           
 "actor_type", - какой живой человек с сердцем пожаловался или авто система (будет либо Citizen, Partner, auto)
 "actor_identifier", - uuid / id актера
 "created_at",
 "updated_at"]

In [10]:
# creating a table from s3

query_text = """--sql 
CREATE TABLE db1.complaints_st_partner
(
    `report_date` Date,
    `issue_id` String,
    `src_type` String,
    `src_uuid` String,
    `src_address_uuid` String,
    `src_address_type` String,
    `device_uuid` String,
    `device_type` String,
    `err_code` String,
    `err_src` String,
    `err_func` String,
    `aasm_state` String,
    `process_error` String,
    `actor_type` String,
    `actor_identifier` String,
    `created_at` DateTime64(3),
    `updated_at` DateTime64(3)
)
ENGINE = S3(
    'https://storage.yandexcloud.net/dwh-asgard/complaints_st_partner/year=*/month=*/*.csv',
    'CSVWithNames'
)
PARTITION BY report_date
SETTINGS date_time_input_format = 'best_effort'
    """

ch.query_run(query_text)

""


In [9]:
query_text = """--sql
    CREATE TABLE db1.complaints_st_partner_ch
(
    `report_date` Date,
    `issue_id` String,
    `src_type` String,
    `src_uuid` String,
    `src_address_uuid` String,
    `src_address_type` String,
    `device_uuid` String,
    `device_type` String,
    `err_code`	String,
    `err_src` String,	
    `err_func`	String,
    `aasm_state` String,	
    `process_error`	String,
    `actor_type` String,
    `actor_identifier` String,
    `created_at` DateTime64,
    `updated_at` DateTime64,
)
    ENGINE = MergeTree()
    ORDER BY src_uuid
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [10]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.complaints_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR RANDOMIZE FOR 1 HOUR TO db1.complaints_st_partner_ch AS
    SELECT
        *
    FROM db1.complaints_st_partner
ORDER BY report_date DESC
"""
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [14]:
query_text = """--sql
    SELECT
        *
    FROM db1.complaints_st_partner_ch
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

,report_date,issue_id,src_type,src_uuid,src_address_uuid,src_address_type,device_uuid,device_type,err_code,err_src,err_func,aasm_state,process_error,actor_type,actor_identifier,created_at,updated_at
0,2025-09-14,,citizen,0254c112-55e6-460f-8cf4-0c3fe7fc8745,a8eaec9c-250b-4ca4-a990-db01ba7c8913,entry,6aa62be4-e631-4804-8e3b-f1ae9c06e0c5,camera,video,MOB,,processed,,auto,,2025-09-14 08:36:58,2025-09-14 08:36:59
1,2025-09-14,,citizen,05f1ccba-3618-43a7-80d0-2e7f314577b7,d0c514a3-4008-4df7-9ad0-4f1d4f995b38,entry,5e46fdf2-8736-4df3-b64c-989325006b43,camera,video,MOB,,processed,,auto,,2025-09-14 13:19:37,2025-09-14 13:19:37
2,2025-09-14,,intercom,05ee4af7-fee4-44ff-8c66-592baabfb076,,,05ee4af7-fee4-44ff-8c66-592baabfb076,intercom,intercom_offline,API,sync,processed,,auto,api,2025-09-14 06:30:17,2025-09-14 06:30:17
3,2025-09-14,,intercom,026200d6-bb50-4946-8a2f-ea1f8b0ef818,0aef0ce6-d2b0-430d-96f0-647334c67a8c,entry,026200d6-bb50-4946-8a2f-ea1f8b0ef818,intercom,intercom_offline,API,sync,processed,,auto,api,2025-09-14 12:30:02,2025-09-14 12:30:03
4,2025-09-14,,citizen,05ea0fe5-3cdf-4bf6-899e-40553f2d1af9,8baa8b7e-3d6f-4de2-bd5f-581e1e29ac9c,entry,5e86b545-5e04-43f8-9045-05a9ef606eef,camera,video,MOB,,processed,,auto,,2025-09-14 15:38:54,2025-09-14 15:38:55
5,2025-09-14,,citizen,02674b34-1c5a-49ca-867a-f750617b1204,4fa53388-ccd3-4f27-839a-39e57732db5e,entry,3dc65928-7933-497f-a3cc-39b3d915adb0,camera,video,MOB,,processed,,auto,,2025-09-14 07:22:16,2025-09-14 07:22:17
6,2025-09-14,,citizen,05e2d1d9-f356-4229-9dfc-c227631f1900,9302b187-ce07-4c98-a295-05647de20041,entry,3c5ca5d8-aa6e-44ee-b437-2a9c674562bc,camera,video,MOB,,processed,,auto,,2025-09-14 14:51:42,2025-09-14 14:51:43
7,2025-09-14,,citizen,027430dc-015f-4f53-bd7f-7359c37312f5,8cbcdfb9-7dfa-47fb-95af-76b1b34f374f,entry,b632cbb2-dda3-4a08-93ce-c5409ce62a79,camera,video,MOB,,processed,,auto,,2025-09-14 12:19:59,2025-09-14 12:20:00
8,2025-09-14,,citizen,027482eb-69fc-41cb-8065-f68da9e71e12,fb85d872-842c-47f2-872d-6384c44f6cdb,entry,7223a5ac-971e-47fe-9bb7-7879a6dd69a0,camera,video,MOB,,processed,,auto,,2025-09-14 09:25:27,2025-09-14 09:25:28
9,2025-09-14,,citizen,05dd8f70-fde8-40eb-888b-ab597d373a65,3c6138c2-973e-4565-a175-dd3ec02a122b,entry,e45c801f-95e3-4241-92d8-6e7c8dc6965b,camera,video,MOB,,processed,,auto,,2025-09-14 06:17:48,2025-09-14 06:17:48


In [ ]:
query_text = """--sql
    SELECT
        *
    FROM db1.complaints_st_partner
    WHERE report_date = '2025-08-16'
    limit 100
    """

ch.query_run(query_text)


DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 27
 Code: 27. DB::Exception: Cannot parse input: expected ',' before: 'UTC,2025-08-17 00:00:26 UTC\n2025-08-17,,intercom,021bbf10-a308-4814-a1f2-aadfc136e5e9,140f09ff-b7d6-4ae0-9380-154e26fb6b8c,entry,021bbf10-a308-4814-a1f2-aadfc13': (at row 1)
: 
Row 1:
Column 0,   name: report_date,      type: Date,          parsed text: "2025-08-17"
Column 1,   name: issue_id,         type: String,        parsed text: <EMPTY>
Column 2,   name: src_type,         type: String,        parsed text: "intercom"
Column 3,   name: src_uuid,         type: String,        parsed text: "fddc8f60-5021-46bc-a3cb-446a7bc7161a"
Column 4,   name: src_address_uuid, type: String,        parsed text: "c7e43007-1daf-45ea-8afc-e8efbf988d66"
Column 5,   name: src_address_type, type: String,        parsed text: "entry"
Column 6,   name: device_uuid,      type: String,        parsed text: "fddc8f60-5021-46bc-

### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.complaints_st_partner_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [7]:
query_text = """--sql
    DROP TABLE db1.complaints_st_partner
    """

ch.query_run(query_text)

""


### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.complaints_st_partner_ch
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.complaints_st_partner_mv
"""

ch.query_run(query_text)